# Face classifier

## Set up

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D 

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score

In [ ]:
seed = 42

## Load data

In [ ]:
data = pd.read_csv("../data/age_gender.csv")

## Baseline model: Multilayer perceptron classifier

In [ ]:
full_img_vec_list = np.array([pxlstring2pxlvec(data, i) for i in range(data.shape[0])])

In [ ]:
target = "gender"
X_train, X_test, y_train, y_test = train_test_split(full_img_vec_list, 
                                                    data[target],
                                                    test_size = 0.2,
                                                    stratify = data[target],
                                                    shuffle = True,
                                                    random_state = seed,
                                                    )

In [ ]:
parameters = {'hidden_layer_sizes':[(32), (64), (128), (256), (512),
                                    (64, 32), (128, 64), (256, 128), (512, 256),
                                    (128, 64, 32), (256, 128, 64), (512, 256, 128)
                                    ],
              'alpha': [0.001, 0.01, 0.1], 
              'max_iter': [10, 50, 100], 
              'learning_rate_init':[0.001, 0.01, 0.1],
              'batch_size': [16, 32]}

base_model = MLPClassifier(activation='relu', 
                           solver='adam', 
                           tol = 0.01,
                           learning_rate='adaptive',
                           verbose = True,
                           shuffle = True,
                           n_iter_no_change = 5,
                           random_state = seed)

cv_model = RandomizedSearchCV(estimator=base_model, 
                              param_distributions=parameters,
                              cv = 3,
                              random_state = seed,
                              verbose = True,
                              n_iter = 5)

cv_model.fit(X_train, y_train)

In [ ]:
plt.plot(cv_model.best_estimator_.loss_curve_)

In [ ]:
cv_model.best_score_

In [ ]:
base_best_params = pd.DataFrame({k: str(v) for k,v in cv_model.best_params_.items()}, index=[0])
base_best_params.to_csv("../results/base_model__best_params.csv", index=False)

In [ ]:
y_pred = cv_model.best_estimator_.predict(X_test)

In [ ]:
#accuracy_score(y_test, y_pred)
pd.DataFrame(classification_report(y_test, y_pred, output_dict=True))

In [ ]:
# Exploiting universal approximation theorem (UAT) for wide, shallow networks
wide_shallow = [(2**n,) for n in range(7,15)]

# Exploiting universal approximation theorem (UAT) for narrow, deep networks
narrow_deep  = [tuple(i*[64]) for i in range(3,11)]

parameters = {'hidden_layer_sizes': wide_shallow + narrow_deep,
              'alpha': [0.001, 0.01, 0.1], 
              'max_iter': [10, 50, 100], 
              'learning_rate_init':[0.001, 0.01, 0.1],
              'batch_size': [16, 32]}

base_model = MLPClassifier(activation='relu', 
                           solver='adam', 
                           tol = 0.01,
                           learning_rate='adaptive',
                           verbose = True,
                           shuffle = True,
                           n_iter_no_change = 5,
                           random_state = seed)

cv_model2 = RandomizedSearchCV(estimator=base_model, 
                              param_distributions=parameters,
                              cv = 3,
                              random_state = seed,
                              verbose = True,
                              n_iter = 10)

cv_model2.fit(X_train, y_train)

In [ ]:
base_best_params2 = pd.DataFrame({k: str(v) for k,v in cv_model2.best_params_.items()}, index=[0])
base_best_params2["score"] = cv_model2.best_score_
base_best_params2.to_csv("../results/base_model2__best_params.csv", index=False)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(1,len(cv_model2.best_estimator_.loss_curve_)+1), cv_model2.best_estimator_.loss_curve_)
ax.set_title("Loss curve")
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss value")
plt.show()

In [ ]:
y_pred = cv_model2.best_estimator_.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
fpr, tpr, thr = roc_curve(y_test, y_pred)

plt.plot(fpr, tpr)
plt.plot([0,1], [0,1], ls=":", c="k")